In [983]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import re

In [984]:
df = pd.read_csv('../../data/raw/hn_th_15102025.csv', encoding='UTF-8')

In [985]:
# Xoá chuyến xe không có giá vé
df.dropna(subset='original_price',axis=0, inplace=True)
# Xoá cột ngày đến nơi
df.drop(columns='arrival_date', inplace=True)

# Xoá cột None
df.drop(columns=['Unnamed: 0', 'Unnamed: 23'], inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)
df

,bus_name,bus_rating,seat_type,departure_time,pick_up_point,arrival_time,drop_of_point,duartion,original_price,sale_price,...,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,36 Limousine,4.8 (502),Limousine 11 chỗ,09:00,• Văn phòng Đồng Tàu,11:30,• Văn phòng Thanh Hóa,2h30m,230.0,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.8,4.8,4.8,4.7,4.7,4.7,4.7
1,Vân Anh Limousine,4.8 (32),Limousine Giường phòng 24 chỗ,13:00,• Bến xe Nước Ngầm,16:00,• Bến xe Thọ Xuân,3h,230.0,200.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.9,4.9,4.9,4.9,4.9,4.9,5.0
2,Thành Trung Limousine,4 (113),Limousine 9 chỗ,07:00,• Văn phòng Hà Nội,09:00,• Văn Phòng Thanh Hóa,2h,230.0,210.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
3,Vĩnh Quang,4.8 (25),Limousine 10 chỗ,11:25,• VP Linh Đàm,13:55,• VP Thanh Hóa - 143 Lê Hồng Phong,2h30m,170.0,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,5.0,4.8,4.9,4.9,4.9,4.9,4.8
4,Thơm Phụng,4.6 (18),Giường nằm 46 chỗ,15:30,• Bến xe Nước Ngầm,17:45,• Big C Thanh Hóa,2h15m,150.0,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.9,4.6,4.8,4.8,4.6,4.5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Thành Trung Limousine,4 (113),Limousine 11 chỗ,05:15,• Sân bay Nội Bài - Sảnh nội địa,08:00,• Văn Phòng Thanh Hóa,2h45m,480.0,460.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
408,Thành Trung Limousine,4 (113),Limousine 11 chỗ,07:15,• Sân bay Nội Bài - Sảnh nội địa,10:00,• Văn Phòng Thanh Hóa,2h45m,480.0,460.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
409,Thành Trung Limousine,4 (113),Limousine 11 chỗ,04:15,• Sân bay Nội Bài - Sảnh nội địa,07:00,• Văn Phòng Thanh Hóa,2h45m,480.0,460.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
410,Thành Trung Limousine,4 (113),Limousine 11 chỗ,04:15,• Sân bay Nội Bài - Sảnh nội địa,07:30,• Văn phòng Nghi Sơn,3h15m,530.0,510.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3


In [986]:
#  Extract feature rating_overall & reviewer_count from bus_raing
def handle_rating_cols(df):
    df['rating_overall'] = df['bus_rating'].str.split(' ', expand=True)[0]
    df['review_count'] = df['bus_rating'].str.split(' ', expand=True)[1].replace(r'[()]', '', regex=True)

    # Xoác cột cũ
    df.drop(columns='bus_rating', axis=1, inplace=True)
    
    # Xoá chuyễn xe không có đánh giá
    df.drop(index=df[df['review_count'] == '0'].index,axis = 0, inplace=True)

    return df

In [987]:
# Extract number_of_seat from seat_type
def handle_seat(df):
    df['number_of_seat'] = df['seat_type'].replace(r'[^0-9]', '', regex=True)

    # Xoá cột cũ
    df.drop(columns='seat_type', inplace=True)

    return df

In [988]:
def handle_price(df):

    price = df[['original_price', 'sale_price']].astype('Int64')
    price.fillna(0, inplace=True)
    
    # Nomalize price / Create new cols
    df['price_original'] = price['original_price'] * 1000
    df['price_discounted'] = price['sale_price'] * 1000

    # Xoá cột cũ
    df.drop(columns=['sale_price', 'original_price'], inplace=True)

    return df

In [989]:
# Nomalize percentage discount
def percent(df):
    percent = df['percent_discount'].replace(r'[-%]', '', regex=True).astype(float)
    percent.fillna(0.0, inplace=True)
    df['discounted_percentage'] = percent

    # Xoá cột cũ
    df.drop(columns='percent_discount', inplace=True)

    return df

In [990]:
def to_time(df:pd.DataFrame, cols_name:str) -> pd.DataFrame:
    df[cols_name] = pd.to_datetime(df[cols_name], format='%H:%M').dt.time
    return df


In [991]:
# Map thứ tiếng Việt sang tiếng Anh
def vn_to_english(df, cols):
    days = {
    'T2': 'Monday',
    'T3': 'Tuesday',
    'T4': 'Wednesday',
    'T5': 'Thursday',
    'T6': 'Friday',
    'T7': 'Saturday',
    'CN': 'Sunday' 
}
    day = df[cols].str.split(', ', expand=True)[0]
    date = df[cols].str.split(', ', expand=True)[1]

    day = day.map(days)

    df['day'] = day
    df['date'] = date 
    df['date'].replace('/', '-',regex=True, inplace=True)
    
    df.drop(columns=cols, inplace=True)
    
    return df

In [992]:
# Cần cải tiến nếu thời gian di chuyển chỉ trong vòng phút (< 1 giờ)
def handle_time(df):
    hours = df['duartion'].str.split('h', expand=True)[0]
    mins = df['duartion'].str.split('h', expand=True)[1]

    mins = mins.replace('m', '', regex=True).replace('', None, regex=True)

    hours = hours.astype(int)
    mins = mins.astype('Int64')

    mins.fillna(0, inplace=True)

    df['duration_m'] = hours * 60 + mins
    
    df.drop(columns='duartion', inplace=True)

    return df

In [993]:
def handle_bus_name(df):

    df['bus_name'].replace(r'\s*\([^)]+\)', '', regex=True, inplace=True)
    return df

In [994]:
def handle_abbr_word(text: str):
    abbr = {
    'vp': 'văn phòng',
    'bx': 'bến xe'
    }
    if not isinstance(text, str):
        return ''
    text = text.lower()
    for key, value in abbr.items():

        text = text.replace(key, value)

    return text

In [995]:
def transform_point(text):

    if not isinstance(text, str):
        return 'Other'
    if re.search(r'\s*văn\s*phòng', text):
        return 'Văn phòng'
    elif re.search(r'\s*bến\s*xe', text):
        return 'Bến xe'
    return 'Other'

In [996]:
df

,bus_name,bus_rating,seat_type,departure_time,pick_up_point,arrival_time,drop_of_point,duartion,original_price,sale_price,...,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,36 Limousine,4.8 (502),Limousine 11 chỗ,09:00,• Văn phòng Đồng Tàu,11:30,• Văn phòng Thanh Hóa,2h30m,230.0,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.8,4.8,4.8,4.7,4.7,4.7,4.7
1,Vân Anh Limousine,4.8 (32),Limousine Giường phòng 24 chỗ,13:00,• Bến xe Nước Ngầm,16:00,• Bến xe Thọ Xuân,3h,230.0,200.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.9,4.9,4.9,4.9,4.9,4.9,5.0
2,Thành Trung Limousine,4 (113),Limousine 9 chỗ,07:00,• Văn phòng Hà Nội,09:00,• Văn Phòng Thanh Hóa,2h,230.0,210.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
3,Vĩnh Quang,4.8 (25),Limousine 10 chỗ,11:25,• VP Linh Đàm,13:55,• VP Thanh Hóa - 143 Lê Hồng Phong,2h30m,170.0,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,5.0,4.8,4.9,4.9,4.9,4.9,4.8
4,Thơm Phụng,4.6 (18),Giường nằm 46 chỗ,15:30,• Bến xe Nước Ngầm,17:45,• Big C Thanh Hóa,2h15m,150.0,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.9,4.6,4.8,4.8,4.6,4.5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Thành Trung Limousine,4 (113),Limousine 11 chỗ,05:15,• Sân bay Nội Bài - Sảnh nội địa,08:00,• Văn Phòng Thanh Hóa,2h45m,480.0,460.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
408,Thành Trung Limousine,4 (113),Limousine 11 chỗ,07:15,• Sân bay Nội Bài - Sảnh nội địa,10:00,• Văn Phòng Thanh Hóa,2h45m,480.0,460.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
409,Thành Trung Limousine,4 (113),Limousine 11 chỗ,04:15,• Sân bay Nội Bài - Sảnh nội địa,07:00,• Văn Phòng Thanh Hóa,2h45m,480.0,460.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
410,Thành Trung Limousine,4 (113),Limousine 11 chỗ,04:15,• Sân bay Nội Bài - Sảnh nội địa,07:30,• Văn phòng Nghi Sơn,3h15m,530.0,510.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3


In [997]:
def handle_rating(df):
    
    rating_cols = [
        'An toàn', 'Thông tin chính xác', 'Thông tin đầy đủ',
        'Thái độ nhân viên', 'Tiện nghi & thoải mái',
        'Chất lượng dịch vụ', 'Đúng giờ'
    ]

    # Điền giá trị thiếu cho các cột rating theo trung bình của từng nhà xe
    df[rating_cols] = (
        df.groupby('bus_name')[rating_cols]
          .transform(lambda x: x.fillna(x.mean().round(1)))
    )

    # Nếu vẫn còn NaN (trường hợp nhà xe không có rating)
    df[rating_cols] = df[rating_cols].fillna(df[rating_cols].mean())

    return df

In [998]:
def preprocess_df(df):
    """
    Tiền xử lý toàn bộ dữ liệu xe khách:
    - Chuẩn hoá rating, giá, thời gian, tên nhà xe
    - Làm sạch điểm đón/trả
    - Điền giá trị thiếu cho rating theo trung bình của từng nhà xe
    """

    

    # Xử lý dữ liệu
    df = handle_rating_cols(df)
    df = handle_seat(df)
    df = handle_price(df)
    df = percent(df)

    # Chuẩn hoá thời gian
    df = to_time(df, 'departure_time')
    df = to_time(df, 'arrival_time')
    df = vn_to_english(df, 'departure_date')
    df = handle_time(df)

    # Chuẩn hoá tên nhà xe
    df = handle_bus_name(df)

    # Làm sạch điểm đón/trả
    df['pick_up_point'] = (
        df['pick_up_point']
        .apply(handle_abbr_word)
        .apply(transform_point)
    )
    df['drop_of_point'] = (
        df['drop_of_point']
        .apply(handle_abbr_word)
        .apply(transform_point)
    )

    df = handle_rating(df)

    return df


In [999]:
df = preprocess_df(df)
df[['number_of_seat', 'review_count']] = df[['number_of_seat', 'review_count']].astype('Int64')
df['rating_overall'] = df['rating_overall'].astype(float)

In [1000]:
df

,bus_name,departure_time,pick_up_point,arrival_time,drop_of_point,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,...,Đúng giờ,rating_overall,review_count,number_of_seat,price_original,price_discounted,discounted_percentage,day,date,duration_m
0,36 Limousine,09:00:00,Văn phòng,11:30:00,Văn phòng,Hà Nội,Thanh Hóa,4.8,4.8,4.8,...,4.7,4.8,502,11,230000,0,0.0,Wednesday,15-10-2025,150
1,Vân Anh Limousine,13:00:00,Bến xe,16:00:00,Bến xe,Hà Nội,Thanh Hóa,4.9,4.9,4.9,...,5.0,4.8,32,24,230000,200000,13.0,Wednesday,15-10-2025,180
2,Thành Trung Limousine,07:00:00,Văn phòng,09:00:00,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,...,4.3,4.0,113,9,230000,210000,9.0,Wednesday,15-10-2025,120
3,Vĩnh Quang,11:25:00,Văn phòng,13:55:00,Văn phòng,Hà Nội,Thanh Hóa,5.0,4.8,4.9,...,4.8,4.8,25,10,170000,0,0.0,Wednesday,15-10-2025,150
4,Thơm Phụng,15:30:00,Bến xe,17:45:00,Other,Hà Nội,Thanh Hóa,4.9,4.6,4.8,...,5.0,4.6,18,46,150000,0,0.0,Wednesday,15-10-2025,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Thành Trung Limousine,05:15:00,Other,08:00:00,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,...,4.3,4.0,113,11,480000,460000,4.0,Wednesday,15-10-2025,165
408,Thành Trung Limousine,07:15:00,Other,10:00:00,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,...,4.3,4.0,113,11,480000,460000,4.0,Wednesday,15-10-2025,165
409,Thành Trung Limousine,04:15:00,Other,07:00:00,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,...,4.3,4.0,113,11,480000,460000,4.0,Wednesday,15-10-2025,165
410,Thành Trung Limousine,04:15:00,Other,07:30:00,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,...,4.3,4.0,113,11,530000,510000,4.0,Wednesday,15-10-2025,195


In [1001]:
df.columns

Index(['bus_name', 'departure_time', 'pick_up_point', 'arrival_time',
       'drop_of_point', 'start_point', 'destination', 'An toàn',
       'Thông tin chính xác', 'Thông tin đầy đủ', 'Thái độ nhân viên',
       'Tiện nghi & thoải mái', 'Chất lượng dịch vụ', 'Đúng giờ',
       'rating_overall', 'review_count', 'number_of_seat', 'price_original',
       'price_discounted', 'discounted_percentage', 'day', 'date',
       'duration_m'],
      dtype='object')

In [1002]:
df[['bus_name','number_of_seat','day', 'date', 'start_point','destination', 'pick_up_point', 'drop_of_point','departure_time', 'arrival_time', 'duration_m','price_original', 'price_discounted', 'discounted_percentage', 'rating_overall', 'review_count', 'An toàn', 'Thông tin chính xác', 'Thông tin đầy đủ',
       'Thái độ nhân viên', 'Tiện nghi & thoải mái', 'Chất lượng dịch vụ',
       'Đúng giờ']]

,bus_name,number_of_seat,day,date,start_point,destination,pick_up_point,drop_of_point,departure_time,arrival_time,...,discounted_percentage,rating_overall,review_count,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,36 Limousine,11,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Văn phòng,Văn phòng,09:00:00,11:30:00,...,0.0,4.8,502,4.8,4.8,4.8,4.7,4.7,4.7,4.7
1,Vân Anh Limousine,24,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Bến xe,Bến xe,13:00:00,16:00:00,...,13.0,4.8,32,4.9,4.9,4.9,4.9,4.9,4.9,5.0
2,Thành Trung Limousine,9,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Văn phòng,Văn phòng,07:00:00,09:00:00,...,9.0,4.0,113,4.4,3.9,4.0,4.3,4.3,3.9,4.3
3,Vĩnh Quang,10,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Văn phòng,Văn phòng,11:25:00,13:55:00,...,0.0,4.8,25,5.0,4.8,4.9,4.9,4.9,4.9,4.8
4,Thơm Phụng,46,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Bến xe,Other,15:30:00,17:45:00,...,0.0,4.6,18,4.9,4.6,4.8,4.8,4.6,4.5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Thành Trung Limousine,11,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Other,Văn phòng,05:15:00,08:00:00,...,4.0,4.0,113,4.4,3.9,4.0,4.3,4.3,3.9,4.3
408,Thành Trung Limousine,11,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Other,Văn phòng,07:15:00,10:00:00,...,4.0,4.0,113,4.4,3.9,4.0,4.3,4.3,3.9,4.3
409,Thành Trung Limousine,11,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Other,Văn phòng,04:15:00,07:00:00,...,4.0,4.0,113,4.4,3.9,4.0,4.3,4.3,3.9,4.3
410,Thành Trung Limousine,11,Wednesday,15-10-2025,Hà Nội,Thanh Hóa,Other,Văn phòng,04:15:00,07:30:00,...,4.0,4.0,113,4.4,3.9,4.0,4.3,4.3,3.9,4.3


In [1003]:
rename_mapping = {
    'bus_name': 'bus_company_name',
    'number_of_seat': 'number_of_seat',  # Tên đã giống
    'date': 'departure_date',
    'departure_time': 'departure_time', # Tên đã giống
    'arrival_time': 'arrival_time',     # Tên đã giống
    'duration_m': 'duration_h_m',
    'price_original': 'price_original', # Tên đã giống
    'price_discounted': 'price_discounted', # Tên đã giống
    'discounted_percentage': 'discount_percentage',
    'rating_overall': 'rating_overall', # Tên đã giống
    'review_count': 'reviewer_count',
    # Các cột rating chi tiết
    'An toàn': 'rating_safety',
    'Thông tin chính xác': 'rating_info_accuracy',
    'Thông tin đầy đủ': 'rating_service_quanity', # Giả định 'Thông tin đầy đủ' tương đương với 'rating_service_quanity'
    'Thái độ nhân viên': 'rating_staff_attitude',
    'Tiện nghi & thoải mái': 'rating_comfort',
    'Chất lượng dịch vụ': 'rating_service_quanity',
    'Đúng giờ': 'rating_punctuality',
    # Các cột khác (có thể cần xử lý riêng nếu muốn nhập vào các bảng khác)
    'start_point': 'city_name',        # (Nếu đây là tên thành phố) -> Dùng để JOIN/LOOKUP trong bảng 'cities'
    'destination': 'city_name',        # (Nếu đây là tên thành phố) -> Dùng để JOIN/LOOKUP trong bảng 'cities'
    'pick_up_point': 'pickup_point',   # Thiếu trong trips, có thể ánh xạ sang 'pickup_point'
    'drop_of_point': 'dropoff_point',  # Thiếu trong trips, có thể ánh xạ sang 'dropoff_point'
    'day': 'day'                        # Không có cột tương đương trực tiếp trong schema
}

In [1004]:
df.drop_duplicates(inplace=True, keep='first')

In [1005]:
df.to_csv('../../data/clean/hn_th_15_10_2025.csv', index=False)

In [1006]:
df_rename = df.rename(rename_mapping)
df1 = df_rename

In [1007]:
df = pd.read_csv('../../data/raw/sg_nt_11102025.csv')
df.drop(columns=['notification', 'Unnamed: 20'], inplace=True)

df.rename(columns={'bus_star':'bus_rating', 'bus_type':'seat_type', 'price_original':'original_price', 'price_discount':'sale_price', 'discount_percent': 'percent_discount', 'duration_h_m':'duartion','bus_company':'bus_name', 'drop_off_point':'drop_of_point'} ,inplace=True)
df['original_price'] = df['original_price'].astype(str).replace(r'\.', '',regex=True).replace('Từ', '', regex=True)
df['sale_price'] = df['sale_price'].astype(str).replace(r'\.', '', regex=True).replace('Từ', '', regex=True)

# df[['original_price','sale_price']] = df[['original_price','sale_price']].astype('Int64')

In [1008]:
df['departure_date'] = 'T7, 11/10/2025'
df

,bus_name,bus_rating,seat_type,departure_time,pick_up_point,duartion,arrival_time,drop_of_point,arrival_date,original_price,sale_price,percent_discount,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ,departure_date
0,Đà Lạt ơi,4.8 (3425),Limousine 24 Phòng ĐÔI,23:30,• Trạm Hàng Xanh,7h,06:30,• Trạm Nha Trang,(12/10),450000,320000,-29%,4.8,4.8,4.8,4.8,4.8,4.8,4.8,"T7, 11/10/2025"
1,Bình Minh Tải,4.7 (3650),Limousine 22 Phòng Đơn,22:30,• Văn Phòng Quận 1,7h5m,05:35,• Văn phòng Nha Trang,(12/10),350000,300000,-14%,4.6,4.6,4.7,4.6,4.6,4.6,4.8,"T7, 11/10/2025"
2,Huỳnh Gia,4.7 (8534),Giường nằm 38 chỗ (WC),22:30,• Văn Phòng Phạm Ngũ Lão,6h30m,05:00,• Văn Phòng Nha Trang,(12/10),280000,250000,-11%,4.8,4.7,4.7,4.7,4.6,4.6,4.7,"T7, 11/10/2025"
3,An Anh Limousine,4.8 (8322),Limousine 34 Phòng Đơn,23:30,• Văn Phòng Quận 5,6h30m,06:00,• Văn phòng Nha Trang,(12/10),299000,199000,-33%,4.8,4.7,4.7,4.7,4.7,4.7,4.6,"T7, 11/10/2025"
4,Khanh Phong,4.7 (16845),Limousine 20 giường phòng (WC),05:15,• Văn Phòng Phạm Ngũ Lão - Quận 1.,6h10m,11:25,• Văn Phòng Nha Trang (KS Mường Thanh),NaN,480000,400000,-17%,4.8,4.7,4.7,4.7,4.6,4.6,4.8,"T7, 11/10/2025"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:30,• Văn Phòng Sài Gòn,7h40m,06:10,• Chợ Vĩnh Lương,(12/10),480000,480000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"T7, 11/10/2025"
561,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:45,• Trạm Phúc An Express,7h15m,06:00,• Văn Phòng Nha Trang,(12/10),470000,470000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"T7, 11/10/2025"
562,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:45,• Trạm Phúc An Express,7h25m,06:10,• Chợ Vĩnh Lương,(12/10),480000,480000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"T7, 11/10/2025"
563,Quỳnh Nhật,3.7 (279),Limousine 24 phòng,10:00,• Bến xe Miền Đông mới,8h5m,18:05,• Ngã 3 Thành.,NaN,650000,650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"T7, 11/10/2025"


In [1009]:
df = preprocess_df(df)
df.drop(columns='arrival_date', inplace=True)
df.drop_duplicates(inplace=True, keep='first')

In [1010]:
df['start_point'] = 'Sài Gòn'
df['destination'] = 'Nha Trang - Khánh Hoà'
df.to_csv('../../data/clean/sg_nt_11_10_2025.csv', index=False)

In [ ]:
import pandas as pd

# Đọc 2 file và đổi tên DataFrame
df_hn_th = pd.read_csv("../../data/clean/hn_th_15_10_2025.csv")
df_sg_nt = pd.read_csv("../../data/clean/sg_nt_11_10_2025.csv")

# (Tùy chọn) Thêm cột 'route' để phân biệt nguồn dữ liệu
df_hn_th['route'] = 'Hà Nội - Thanh Hoá'
df_sg_nt['route'] = 'Sài Gòn - Nha Trang'

# Gộp lại thành một DataFrame
df_all = pd.concat([df_hn_th, df_sg_nt], ignore_index=True)

# Kiểm tra kích thước sau khi gộp
print("Số dòng sau khi gộp:", len(df_all))

# Lưu file hợp nhất
df_all.to_csv("all_routes_merged.csv", index=False)
df_all


Số dòng sau khi gộp: 936


,bus_name,departure_time,pick_up_point,arrival_time,drop_of_point,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,...,rating_overall,review_count,number_of_seat,price_original,price_discounted,discounted_percentage,day,date,duration_m,route
0,36 Limousine,09:00:00,Văn phòng,11:30:00,Văn phòng,Hà Nội,Thanh Hóa,4.800000,4.80000,4.800000,...,4.8,502,11,230000,0,0.0,Wednesday,15-10-2025,150,Hà Nội - Thanh Hoá
1,Vân Anh Limousine,13:00:00,Bến xe,16:00:00,Bến xe,Hà Nội,Thanh Hóa,4.900000,4.90000,4.900000,...,4.8,32,24,230000,200000,13.0,Wednesday,15-10-2025,180,Hà Nội - Thanh Hoá
2,Thành Trung Limousine,07:00:00,Văn phòng,09:00:00,Văn phòng,Hà Nội,Thanh Hóa,4.400000,3.90000,4.000000,...,4.0,113,9,230000,210000,9.0,Wednesday,15-10-2025,120,Hà Nội - Thanh Hoá
3,Vĩnh Quang,11:25:00,Văn phòng,13:55:00,Văn phòng,Hà Nội,Thanh Hóa,5.000000,4.80000,4.900000,...,4.8,25,10,170000,0,0.0,Wednesday,15-10-2025,150,Hà Nội - Thanh Hoá
4,Thơm Phụng,15:30:00,Bến xe,17:45:00,Other,Hà Nội,Thanh Hóa,4.900000,4.60000,4.800000,...,4.6,18,46,150000,0,0.0,Wednesday,15-10-2025,135,Hà Nội - Thanh Hoá
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,Phúc An Express,22:30:00,Văn phòng,06:10:00,Other,Sài Gòn,Nha Trang - Khánh Hoà,4.800000,4.70000,4.700000,...,4.7,3425,22,480000000,480000000,0.0,Saturday,11-10-2025,460,Sài Gòn - Nha Trang
932,Phúc An Express,22:45:00,Other,06:00:00,Văn phòng,Sài Gòn,Nha Trang - Khánh Hoà,4.800000,4.70000,4.700000,...,4.7,3425,22,470000000,470000000,0.0,Saturday,11-10-2025,435,Sài Gòn - Nha Trang
933,Phúc An Express,22:45:00,Other,06:10:00,Other,Sài Gòn,Nha Trang - Khánh Hoà,4.800000,4.70000,4.700000,...,4.7,3425,22,480000000,480000000,0.0,Saturday,11-10-2025,445,Sài Gòn - Nha Trang
934,Quỳnh Nhật,10:00:00,Bến xe,18:05:00,Other,Sài Gòn,Nha Trang - Khánh Hoà,4.492763,4.36864,4.431579,...,3.7,279,24,650000000,650000000,0.0,Saturday,11-10-2025,485,Sài Gòn - Nha Trang


In [1017]:
df['number_of_seat'].value_counts()


number_of_seat
21    123
32    121
24    120
34     67
22     57
20     36
40     18
4       8
38      4
46      2
36      2
44      2
30      1
41      1
Name: count, dtype: int64

In [1018]:
df_all.to_csv("all_routes_merged.csv", index=False)